In [4]:
import json

from pyspark.sql import SparkSession
from pymongo import MongoClient

In [2]:
client = MongoClient('mongo', 27017)

In [3]:
db = client.test
collection = db.northwind

In [12]:
with open('/workspace/assignments/data/northwind.json') as f:
    file_data = json.load(f)
    result = collection.insert_one(file_data)

Inserted 644238e4c85f02bcceec201f document


In [13]:
client.close()

In [14]:
packages = {
   'org.mongodb.spark:mongo-spark-connector_2.12:10.1.1',
}

In [20]:
spark = (
    SparkSession.builder
    .config('spark.jars.packages', ','.join(packages))
    .config("spark.mongodb.input.uri", "mongodb://mongo:27017/lotr.heroes")
    .config("spark.mongodb.output.uri", "mongodb://mongo:27017/lotr.heroes")
    .config("spark.mongodb.connection.uri", "mongodb://mongo:27017")
    .config("spark.mongodb.database", "lotr")
    .config("spark.mongodb.collection", "heroes")
    .getOrCreate()
)

23/04/21 07:22:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [21]:
people = spark.createDataFrame(
    [
        ("Bilbo Baggins", 50),
        ("Gandalf", 1000),
        ("Thorin", 195),
        ("Balin", 178),
        ("Kili", 77),
        ("Dwalin", 169),
        ("Oin", 167),
        ("Gloin", 158),
        ("Fili", 82),
        ("Bombur", None),
    ],
    ["name", "age"],
)


In [22]:
people.write.format("mongodb").mode("overwrite").save()

23/04/21 07:22:34 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/21 07:22:49 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/21 07:23:04 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/21 07:23:19 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


ERROR:root:KeyboardInterrupt while sending command.                 (0 + 0) / 2]
Traceback (most recent call last):
  File "/home/hduser/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/hduser/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

23/04/21 07:23:34 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [10]:
df = spark.read.format("mongodb").load()
df.printSchema()
df.show()

root
 |-- _id: string (nullable = true)
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

+--------------------+----+-------------+
|                 _id| age|         name|
+--------------------+----+-------------+
|64305f42a842da159...|1000|      Gandalf|
|64305f42a842da159...|null|       Bombur|
|64305f42a842da159...|  77|         Kili|
|64305f42a842da159...| 158|        Gloin|
|64305f42a842da159...| 195|       Thorin|
|64305f42a842da159...| 178|        Balin|
|64305f42a842da159...|  50|Bilbo Baggins|
|64305f42a842da159...|  82|         Fili|
|64305f42a842da159...| 169|       Dwalin|
|64305f42a842da159...| 167|          Oin|
+--------------------+----+-------------+



In [ ]:
df = spark.read.format("mongodb").options(database="test", collection="northwind").load()